# Text Classification and Analysis On Extracted Data from website articles.

# Importing Data

In [ ]:
import pandas as pd

input_data = pd.read_excel('/kaggle/input/website-urls-for-web-scraping-and-data-analysis/blackcoffer websites link dataset.xlsx')
print(input_data)

# Web Scraping

Creating a function to extract article title and text content using url of article

In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_data_in_txt(url, url_id):

  # Making a GET request for the given article link
  r = requests.get(url)

  # Parsing the HTML
  soup = BeautifulSoup(r.content, 'html.parser')
  
  # Finding the title and all div elements to get article contents (text only)
  title = soup.find('title')
  div = soup.find_all('div',  class_ = 'td-post-content tagdiv-type') 

  # Creating a text file with particular url's id as file name
  filename = str(url_id)
  file =  open(filename + ".txt", "w")  #Opening file in write mode

  
  file.write(title.text.strip())  # Removing extra spaces from the title and convertion in lower case before writing in file
  file.write(".")

  for i in div:
    lines = i.find_all('p') #finding all 'p: paragraph' type attributes of article

    # extracting text data line by line and writing into a file
    for line in lines:
      file.write(line.text.strip())  # Removing extra spaces from the text before writing in file
      file.write(".")

  file.close() # Close the file after writing all the text content of an article

#example
extract_data_in_txt(input_data.URL[0], input_data.URL_ID[0])
filename = str(input_data.URL_ID[0])
file =  open(filename + ".txt", "r") 
print(file.read())

# Creating text files for each article 
Each article's title and content is stored in a text file as url_id its filename.

In [ ]:
def create_files():
  for i in range(len(input_data.URL)):
    extract_data_in_txt(input_data.URL[i], input_data.URL_ID[i])

create_files()

# Sentiment Analysis

# Creating a list of stop words

In [ ]:
import re

#filenames for all files containing stop words
path = "/kaggle/input/website-urls-for-web-scraping-and-data-analysis/StopWords_"

auditor = path + "Auditor.txt"
currency = path + "Currencies.txt"
numbers = path + "DatesandNumbers.txt"
generic = path + "Generic.txt"
generic_long = path + "GenericLong.txt"
geographic = path + "Geographic.txt"
names = path + "Names.txt"

filenames = [auditor, currency, numbers, generic, generic_long, geographic, names]

# Create a list to contain all the stop words
stop_words = []

for fname in filenames:
    df = pd.read_csv(fname, sep = "|", encoding='latin-1', header = None)
    word_list = [str(i).lower().strip() for i in df[0]]
    stop_words.extend(word_list)
    
# printing all the stop words
print(stop_words)

# Creating a dictionary of positive and negative words

In [ ]:
path = "/kaggle/input/website-urls-for-web-scraping-and-data-analysis/"

positive = path + "positive-words.txt"
negative = path + "negative-words.txt"

# Create lists for positive and negative words
pos_words = []
neg_words = []

df = pd.read_csv(positive, sep = " ", encoding='latin-1', header = None)
pos_words = [str(i).lower().strip() for i in df[0] if(str(i).lower().strip() not in stop_words)]
    

df = pd.read_csv(negative, sep = " ", encoding='latin-1', header = None)
neg_words = [str(i).lower().strip() for i in df[0] if(str(i).lower().strip() not in stop_words)]

dictionary = {'positive': pos_words,
                      'negative': neg_words}

# printing all the positive and negative words
print(dictionary)

# Data Cleaning

In [ ]:
!pip install contractions

# Expanding contractions

In [ ]:
import contractions

#Expanding contractions such as I'll to I will etc.
def expand_contractions(s):
    return contractions.fix(s)

# Lemmatizing sentence: converting words in their lemma(root) form

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
 
# Create a Doc object
def lemmatize_sentence(sentence):
    doc = nlp(sentence)
    # Create list of tokens from given string
    tokens = []
    for token in doc:
        tokens.append(token)
 
    lemmatized_sentence = " ".join([token.lemma_ for token in doc])
    return lemmatized_sentence 

In [ ]:
def clean_data(clean_sentence):
  
  clean_sentence = expand_contractions(clean_sentence)

  clean_sentence = re.sub(r'[^\w\s]', '', clean_sentence)  #removing punctuations like "/", ";" "[", "]" "=", "#" etc.

  clean_sentence = lemmatize_sentence(clean_sentence)

  return clean_sentence

# Extracting Derived variables

We convert the text into a list of tokens using the nltk tokenize module and use these tokens to calculate the 4 variables described below:

1. **Positive Score**: This score is calculated by assigning the value of +1 for each word if found in the Positive Dictionary and then adding up all the values.
2. **Negative Score**: This score is calculated by assigning the value of -1 for each word if found in the Negative Dictionary and then adding up all the values. We multiply the score with -1 so that the score is a positive number.
3. **Polarity Score**: This is the score that determines if a given text is positive or negative in nature. It is calculated by using the formula: 
Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
Range is from -1 to +1
4. **Subjectivity Score**: This is the score that determines if a given text is objective or subjective. It is calculated by using the formula:Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001). Range is from 0 to +1 


In [ ]:
from nltk import word_tokenize

def extracting_derived_scores(clean_sentence):

  clean_sentence = clean_sentence.lower() 

  clean_sentence = clean_data(clean_sentence)

  clean_sentence = clean_sentence.split(" ")  #tokenization: splitting sentence into words

  clean_lst = []  #removing stop words like if, but, or etc. and removing characters of length 1

  for word in clean_sentence:
    if (word not in stop_words):
      clean_lst.append(word)

  positivity_score = 0
  negativity_score = 0

  for word in clean_lst:
    if(word in pos_words):
      positivity_score += 1
    if(word in neg_words):
      negativity_score += 1
    else:
      continue

  polarity_score = (positivity_score - negativity_score) / ((positivity_score + negativity_score) + 0.000001)
  subjectivity_score = (positivity_score + negativity_score) / ((len(clean_lst)) + 0.000001)

  scores = [positivity_score, negativity_score, polarity_score, subjectivity_score]

  return scores


In [ ]:
#Creating a list filenames, containing filename of every article

path = "/kaggle/working/"

filenames = [path+str(i)+".txt" for i in input_data.URL_ID]

In [ ]:
#storing these 4 scores in seperate lists
#each list contains that particular score for all the text files.

positivity_score = [] 
negativity_score = [] 
polarity_score = [] 
subjectivity_score = []

for fname in filenames:
  file = open(fname, 'r')
  text = file.read()
  score = extracting_derived_scores(text)
  positivity_score.append(score[0])
  negativity_score.append(score[1])
  polarity_score.append(score[2])
  subjectivity_score.append(score[3])


# Average Sentence Length

Sum of sentence lengths / Total no. of sentences

In [ ]:
avg_sentence_len = []  #list to score avg sentence length of each document

for fname in filenames:

  file = open(fname, 'r')
  text = file.read() #extracting all the file content in a string

  sentence_list = text.split(".") #split function will return a list containting sentences seperated by full-stop
  
  sentence_lengths = [len(i) for i in sentence_list]

  sentence_count = len(sentence_list)

  avg_sentence_len.append(sum(sentence_lengths) / sentence_count)


# Average No. of Words per Sentence

Sum of no. of words in a sentence / Total no. of sentences

In [ ]:
avg_words_per_sentence = [] #list to score avg no. of words per sentence for each document

for fname in filenames:

  words_in_sentence_count = []

  file = open(fname, 'r')
  text = file.read() #extracting all the file content in a string

  sentence_list = text.split(".") #split function will return a list containting sentences seperated by full-stop
  
  for i in sentence_list:
    word_count = 0
    for word in i.split(" "):
      word_count += 1
    words_in_sentence_count.append(word_count)
  
  sentence_count = len(sentence_list)

  avg_words_per_sentence.append(sum(words_in_sentence_count) / sentence_count)


# Average Word Length

Sum of lengths of words / total words in a document

In [ ]:
avg_words_length = [] #list to score avg word length of each document

for fname in filenames:
  
  word_lengths = []

  file = open(fname, 'r')
  text = file.read()

  text = re.sub(r'\.', '', text)
  
  text = text.split(" ")

  for i in text:
    word_lengths.append(len(i))
    
  avg_words_length.append(sum(word_lengths) / len(text))

# Word Count after Data Cleaning

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
import re

word_count = []

for fname in filenames:

  file = open(fname, 'r')
  text = file.read()
    
  text = re.sub(r'[^\w\s]', '', text)  #removing punctuations like "/", ";" "[", "]" "=", "#" etc.       
  text = text.split()  #tokenization: splitting sentence into words
    
  stop_words = set(stopwords.words('english'))
  clean_lst = []  #removing stop words like if, but, or etc. and removing characters of length 1
  for word in text:
      if (word not in stop_words):
          if(len(word) != 1):
              clean_lst.append(word)
              
  word_count.append(len(clean_lst))

# Personal Pronouns

In [ ]:
personal_pronouns = re.compile(r'\b(I|we|ours|my|mine|(?-i:us))\b', re.I)
pronoun_count = [] #list to contain the no. of personal pronouns in each document

for fname in filenames:

  file = open(fname, 'r')
  text = file.read()
    
  pronouns = personal_pronouns.findall(text) 

  pronoun_count.append(len(pronouns))
 

# Average Syllable Count per Word

Sum of syllable count in each word / total no. of words

In [ ]:
avg_syllable_count = []

for fname in filenames:

  file = open(fname, 'r')
  text = file.read()
  
  sentence_list = text.split(".") #split function will return a list containting sentences seperated by full-stop
  vowels = 'aeiou'

  text = re.sub(r'\.', '', text)
  
  text = text.split(" ")

  syl_count = 0  
  
  for word in text:
    syllables = re.findall(f'(?!e$)(?!es$)(?!ed$)[{vowels}]', word, re.I)
    syl_count += len(syllables)

  avg_syllable_count.append(syl_count / len(text))
  
 

# Complex Word Count: Word containing two or more syllables

In [ ]:
complex_word_count = []
word_count = []

for fname in filenames:

  file = open(fname, 'r')
  text = file.read()
  
  sentence_list = text.split(".") #split function will return a list containting sentences seperated by full-stop

  text = re.sub(r'\.', '', text)
  
  words_list = text.split(" ")
  word_count.append(len(words_list))

  c = 0
  for word in words_list:
    l = re.findall('(?!e$)[aeiou]+', word, re.I)+re.findall('^[aeiouy]*e$', word, re.I)
    if len(l) > 2:
      c += 1
  complex_word_count.append(c)


# Percentage of Complex words:
 (the number of complex words / the number of words) *100

In [ ]:
complex_words_percent = []
for i in range(len(complex_word_count)):
  complex_words_percent.append((complex_word_count[i]*100)/word_count[i])


# Fog Index:
0.4 * (Average Sentence Length + Percentage of Complex words)

In [ ]:
fog_index = []

for i in range(len(avg_sentence_len)):
  fog_index.append((complex_words_percent[i] + avg_sentence_len[i])*0.4)


# Creating Output DataFrame

In [ ]:
output_data = input_data.copy()

output_data['Positive Score'] = pd.Series(positivity_score)
output_data['Negative Score'] = pd.Series(negativity_score)
output_data['Polarity Score'] = pd.Series(polarity_score)
output_data['Subjectivity Score'] = pd.Series(subjectivity_score)

output_data['Avg Sentence Length'] = pd.Series(avg_sentence_len)
output_data['Avg No of Words per Sentence'] = pd.Series(avg_words_per_sentence)
output_data['Avg Word Length'] = pd.Series(avg_words_length)

output_data['Personal Pronoun Count'] = pd.Series(pronoun_count)
output_data['Word Count'] = pd.Series(word_count)
output_data['Average Syllable Count'] = pd.Series(avg_syllable_count)
output_data['Complex Word Count'] = pd.Series(complex_word_count)

output_data['Percentage of Complex Words'] = pd.Series(complex_words_percent)

output_data['Fog Index'] = pd.Series(fog_index)

print(output_data)
  


# Output data frame to excel sheet

In [ ]:
output_data.to_excel('Output Data Structure.xlsx', index = False)

In [ ]:
output_data.head(10)